# 데이터 준비


In [1]:
# 라이브러리 호출
import pandas as pd

In [2]:
# 원본 데이터 불러오기
df_raw = pd.read_csv("p24v31_KMP_csv.csv", encoding='cp949')
df_raw = df_raw.replace(' ', pd.NA)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_157876\1385228898.py:2: DtypeWarning: Columns (5,8,9,295,296,297,298,299,300,371,372,373,374) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv("p24v31_KMP_csv.csv", encoding='cp949')


### 전처리 과정
##### 1. 사용할 데이터 추출

In [3]:
# 최종 사용할 변수 리스트
selected_vars = [
    'p24gender', 'p24age', 'p24school', 'p24job1', 'p24income',
    'p24d25070',
    'p24n01038', 'p24n01041', 'p24n01042', 'p24n01043',
    'p24d27001', 'p24d27004',
    'p24d26007',
    'p24d31001', 'p24d31002'
]

# 선택변수 추출 데이터
df_selected = df_raw[selected_vars].copy()

# 데이터 타입 변환
for col in ['p24d27001', 'p24d27004', 'p24d26007']:
    df_selected[col] = pd.to_numeric(df_selected[col], errors='coerce')

# 결과 확인
print(df_selected.head())

   p24gender  p24age  p24school  p24job1  p24income  p24d25070  p24n01038  \
0          2       6          5        2          1          5          2   
1          2       3          5        1          5          5          2   
2          1       3          5        1          5          5          2   
3          1       6          5        1          5          5          2   
4          1       4          5        1          5          5          3   

   p24n01041  p24n01042  p24n01043  p24d27001  p24d27004  p24d26007  \
0          2          3          3       11.0        5.0        1.0   
1          2          3          1       11.0        3.0        1.0   
2          2          1          1        3.0        4.0        1.0   
3          2          2          2       11.0        4.0        1.0   
4          2          3          1       11.0        5.0        1.0   

   p24d31001  p24d31002  
0          2          2  
1          2          2  
2          3          2  
3     

##### 2. 추출한 변수명 한글 매핑

In [4]:
# 변수 추출 완료된 데이터 불러오기
df = df_selected.copy()

# 변수명 한글 매핑 딕셔너리 정의
column_mapping = {
    'p24gender': '성별',
    'p24age': '연령',
    'p24school': '학력',
    'p24job1': '직업',
    'p24income': '소득',
    'p24d25070': '검색능력',
    'p24n01038': '정보판별1(사실확인)',
    'p24n01041': '정보판별2(근거확인)',
    'p24n01042': '정보판별3(상업적의도확인)',
    'p24n01043': '정보판별4(정치적의도확인)',
    'p24d27001': 'SNS피드확인',
    'p24d27004': 'SNS좋아요댓글',
    'p24d26007': 'OTT이용빈도',
    'p24d31001': 'AI인지수준',
    'p24d31002': 'AI이용여부'
}

# 변수명 변경 적용
df.rename(columns=column_mapping, inplace=True)

# 결과 확인
print(df.head())

# 데이터 저장
df_rename_kor = df.copy()

   성별  연령  학력  직업  소득  검색능력  정보판별1(사실확인)  정보판별2(근거확인)  정보판별3(상업적의도확인)  \
0   2   6   5   2   1     5            2            2               3   
1   2   3   5   1   5     5            2            2               3   
2   1   3   5   1   5     5            2            2               1   
3   1   6   5   1   5     5            2            2               2   
4   1   4   5   1   5     5            3            2               3   

   정보판별4(정치적의도확인)  SNS피드확인  SNS좋아요댓글  OTT이용빈도  AI인지수준  AI이용여부  
0               3     11.0       5.0      1.0       2       2  
1               1     11.0       3.0      1.0       2       2  
2               1      3.0       4.0      1.0       3       2  
3               2     11.0       4.0      1.0       3       2  
4               1     11.0       5.0      1.0       3       2  


##### 3. 변수 데이터 매핑

In [5]:
# 데이터 불러오기
df = df_rename_kor.copy()

### 이진형 매핑들 (성별, 직업, 스마트폰보유, AI이용여부)

# 성별 (남:0, 여:1)
df['성별'] = df['성별'].map({1: 0, 2: 1})

# 직업 (없음:0, 있음:1)
df['직업'] = df['직업'].map({1: 1, 2: 0})

# AI 이용여부 (없음:0, 있음:1)
df['AI이용여부'] = df['AI이용여부'].map({1: 1, 2: 0})

### 소득 그룹화 (저:0, 중:1, 고:2)
df['소득그룹'] = df['소득'].apply(lambda x: 0 if x <= 3 else (1 if x <= 6 else 2))

### SNS피드확인 매핑 (11단계 → )
sns_feed_mapping = {
    9: 11,  # 하루 5회 이상
    10: 10, # 하루 3~4회
    11: 9, # 하루 1~2회
    1: 8,   # 하루에도 여러 번
    2: 7,
    3: 6,
    4: 5,
    5: 4,
    6: 3,
    7: 2,
    8: 1
}
df['SNS피드확인'] = df['SNS피드확인'].map(sns_feed_mapping)

### SNS좋아요댓글 매핑 (12단계 → 적극성 척도)
sns_mapping = {
    1: 10, 2: 9, 3: 8, 4: 7,
    5: 6, 6: 5, 7: 4, 8: 3,
    9: 2, 10: 12, 11: 11, 12: 10
}
df['SNS좋아요댓글'] = df['SNS좋아요댓글'].map(sns_mapping)

### OTT이용빈도 매핑 (8단계 → 8점 척도)
ott_mapping = {
    1: 8, 2: 7, 3: 6, 4: 5,
    5: 4, 6: 3, 7: 2, 8: 1
}
df['OTT이용빈도'] = df['OTT이용빈도'].map(ott_mapping)

# SNS 비사용자는 활동이 없다고 간주하여 0으로 처리
df['SNS피드확인'] = df['SNS피드확인'].fillna(0)
df['SNS좋아요댓글'] = df['SNS좋아요댓글'].fillna(0)
df['OTT이용빈도'] = df['OTT이용빈도'].fillna(0)

# 리커트형 (검색능력, 정보판별1~4, AI인지수준)은 그대로 사용 (1~5 또는 1~4 점수)

# 결과 확인
print(df.head())

# 데이터 저장
df_mapped = df.copy()

   성별  연령  학력  직업  소득  검색능력  정보판별1(사실확인)  정보판별2(근거확인)  정보판별3(상업적의도확인)  \
0   1   6   5   0   1     5            2            2               3   
1   1   3   5   1   5     5            2            2               3   
2   0   3   5   1   5     5            2            2               1   
3   0   6   5   1   5     5            2            2               2   
4   0   4   5   1   5     5            3            2               3   

   정보판별4(정치적의도확인)  SNS피드확인  SNS좋아요댓글  OTT이용빈도  AI인지수준  AI이용여부  소득그룹  
0               3      9.0       6.0      8.0       2       0     0  
1               1      9.0       8.0      8.0       2       0     1  
2               1      6.0       7.0      8.0       3       0     1  
3               2      9.0       7.0      8.0       3       0     1  
4               1      9.0       6.0      8.0       3       0     1  


##### 4. 소외군 비소외군 라벨 생성

In [6]:
# 전처리 완료된 데이터 불러오기 (최종 매핑본 사용)
df = df_mapped.copy()

# 소외군 라벨 생성 함수 정의
def assign_label(row):
    if row['AI이용여부'] == 1:  # 이용 있음
        return 0    # 정상활용군
    elif row['AI이용여부'] == 0:    # 이용 안함
        if row['AI인지수준'] in [1, 2]:
            return 1    # 소외군
        else:
            return 2    # 보조군
    else:
        return pd.NA    # 예외처리

# 소외군 라벨 생성
df['소외군라벨'] = df.apply(assign_label, axis=1)

# 결과 확인
print(df[['AI인지수준', 'AI이용여부', '소외군라벨']].head())

# 데이터 저장
df_labeled = df.copy()

   AI인지수준  AI이용여부  소외군라벨
0       2       0      1
1       2       0      1
2       3       0      2
3       3       0      2
4       3       0      2


##### 5. 기존 소외군 라벨을 이진화

In [7]:
# 기존 라벨이 포함된 데이터 불러오기
df = df_labeled.copy()

# 이진 라벨 생성 (비소외군:0, 소외군:1)
df['소외이진라벨'] = df['소외군라벨'].apply(lambda x: 1 if x == 1 else 0)

# 결과 확인
print(df[['소외군라벨', '소외이진라벨']].head())

# 데이터 저장
df_binary = df.copy()

   소외군라벨  소외이진라벨
0      1       1
1      1       1
2      2       0
3      2       0
4      2       0


### AI기기 보유율 매핑

In [8]:
# 기존 전처리 완료 데이터 불러오기 (이진라벨 포함 데이터 기준)
df = df_binary.copy()
df = df.dropna(subset=['소외이진라벨'])

### AI기기 보유율 매핑

# (1) 성별
gender_map = {0: 15.3, 1: 7.0}
df['AI기기보유율_성별'] = df['성별'].map(gender_map)

# (2) 연령
age_map = {
    1: None,
    2: 12.9,
    3: 12.9,
    4: 17.9,
    5: 20.0,
    6: 15.8,
    7: 11.4,
    8: 3.0
}
df['AI기기보유율_연령'] = df['연령'].map(age_map)

# (3) 소득
income_map = {
    0: 5.37,
    1: 17.03,
    2: 23.1
}
df['AI기기보유율_소득'] = df['소득그룹'].map(income_map)

# (4) 학력
def map_edu(x):
    if x <= 4:
        return 1
    elif x == 5:
        return 2
    else:
        return 3
df['학력그룹'] = df['학력'].apply(map_edu)

edu_map = {
    1: 9.0,
    2: 19.7,
    3: 0.0
}
df['AI기기보유율_학력'] = df['학력그룹'].map(edu_map)

# 결과 확인
print(df[['AI기기보유율_성별', 'AI기기보유율_연령',
          'AI기기보유율_소득', 'AI기기보유율_학력']].head())

# 데이터 저장
df_binary_AI = df.copy()

   AI기기보유율_성별  AI기기보유율_연령  AI기기보유율_소득  AI기기보유율_학력
0         7.0        15.8        5.37        19.7
1         7.0        12.9       17.03        19.7
2        15.3        12.9       17.03        19.7
3        15.3        15.8       17.03        19.7
4        15.3        17.9       17.03        19.7


### 파생변수 추가

In [9]:
# 기존 전처리 완료 데이터 불러오기
df = df_binary_AI.copy()

### 파생변수 생성

# 디지털 적극성 지수 (Active Score)
df['Active_Score_raw'] = (
    df['SNS피드확인'] + 
    df['SNS좋아요댓글'] + 
    df['OTT이용빈도'] +
    df['검색능력'] 
)

# 정보판별 종합지수 (Information Literacy Index)
df['Info_Literacy_Index_raw'] = (
    df['정보판별1(사실확인)'] + 
    df['정보판별2(근거확인)'] + 
    df['정보판별3(상업적의도확인)'] + 
    df['정보판별4(정치적의도확인)']
)

# AI기기보유율 통합 파생변수 생성 (예: 평균으로 계산)
df['AI_Device_raw'] = df[[
    'AI기기보유율_성별',
    'AI기기보유율_연령',
    'AI기기보유율_소득',
    'AI기기보유율_학력'
]].mean(axis=1)

# Min-Max 정규화 함수
def min_max_norm(series):
    return (series - series.min()) / (series.max() - series.min())

# 정규화 적용
df['Active_Score'] = min_max_norm(df['Active_Score_raw'])
df['Info_Literacy_Index'] = min_max_norm(df['Info_Literacy_Index_raw'])
df['AI_Device'] = min_max_norm(df['AI_Device_raw'])

# 중간 결과 확인
print(df[['Active_Score', 'Info_Literacy_Index', 'AI_Device']].describe())

# 정규화 이전 raw 컬럼은 제거
df.drop(columns=['Active_Score_raw', 'Info_Literacy_Index_raw', 'AI_Device_raw'], inplace=True)

df_features = df.copy()

# 저장
df.to_csv("p24_final_data.csv", index=False, encoding='utf-8-sig')

       Active_Score  Info_Literacy_Index    AI_Device
count   8693.000000          8693.000000  8693.000000
mean       0.428644             0.489719     0.548375
std        0.257701             0.260301     0.238925
min        0.000000             0.000000     0.000000
25%        0.257143             0.312500     0.347521
50%        0.428571             0.500000     0.585047
75%        0.628571             0.687500     0.737446
max        1.000000             1.000000     1.000000
